In [2]:
import kaggle

Dataset URL: https://www.kaggle.com/datasets/ankitbansal06/retail-orders
License(s): CC0-1.0




  0%|          | 0.00/200k [00:00<?, ?B/s]
100%|██████████| 200k/200k [00:00<00:00, 278kB/s]
100%|██████████| 200k/200k [00:00<00:00, 277kB/s]


In [2]:
#Download dataset using Kaggle API
!kaggle datasets download ankitbansal06/retail-orders -f orders.csv

Dataset URL: https://www.kaggle.com/datasets/ankitbansal06/retail-orders
License(s): CC0-1.0
orders.csv.zip: Skipping, found more recently modified local copy (use --force to force download)


In [3]:
#Extract file from zip file
import zipfile
zip_ref = zipfile.ZipFile("orders.csv.zip")
zip_ref.extractall()
zip_ref.close()    

In [12]:
#Read data from the files and handle Null values
import pandas as pd

df = pd.read_csv('orders.csv',na_values = ['Not Available', 'unknown'])

In [13]:
df['Ship Mode'].unique()


array(['Second Class', 'Standard Class', nan, 'First Class', 'Same Day'],
      dtype=object)

In [24]:
#rename columns names ..make them lower case and replace space with underscore
# df.rename(columns={"Order Id":"order_id"})
# df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(" ","_")
df.head(5)


,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,cost_price,list_price,quantity,discount_percent
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,780,960,5,2
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,20,20,2,5


In [30]:
#derive new columns discount , sale price and profit
# df["discount"] = df["list_price"]*df["discount_percent"]*.01
# df["sale_price"] = df['list_price'] - df["discount"]
df["profit"] = df["sale_price"] - df["cost_price"]

In [35]:
df.head(2)

,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,cost_price,list_price,quantity,discount_percent,discount,sale_price,profit
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2,5.2,254.8,14.8
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3,21.9,708.1,108.1


In [34]:
#convert order date from object data type to datetime
df["order_date"] = pd.to_datetime(df['order_date'],format='%Y-%m-%d')

In [36]:
df.dtypes

order_id                     int64
order_date          datetime64[ns]
ship_mode                   object
segment                     object
country                     object
city                        object
state                       object
postal_code                  int64
region                      object
category                    object
sub_category                object
product_id                  object
cost_price                   int64
list_price                   int64
quantity                     int64
discount_percent             int64
discount                   float64
sale_price                 float64
profit                     float64
dtype: object

In [ ]:
#drop cost price list price and discount percent columns
df.drop(columns=['cost_price','list_price','discount_percent'],inplace=True)

In [1]:
import sqlalchemy as sal

# Replace the placeholders with your actual MySQL connection details
username = ''
password = ''
host = ''  # or the IP address of your MySQL server
port =          # Default MySQL port
database = 'orders_analysis1'

# Connection string
connection_string = f"mysql+pymysql://{username}:{password}@{host}:{port}/{database}"

# Create the engine and connect
engine = sal.create_engine(connection_string)
conn = engine.connect()

# Test the connection
try:
    print("Connection successful!")
except Exception as e:
    print("Connection failed:", e)
# finally:
#     conn.close()


Connection successful!


In [53]:
#load the data into sql server using append option
df.to_sql('df_orders', con=conn , index=False, if_exists = 'append')

9994

In [54]:
df.columns

Index(['order_id', 'order_date', 'ship_mode', 'segment', 'country', 'city',
       'state', 'postal_code', 'region', 'category', 'sub_category',
       'product_id', 'quantity', 'discount', 'sale_price', 'profit'],
      dtype='object')

In [56]:
row_count = len(df)
print(row_count)

9994
